<a href="https://colab.research.google.com/github/SSenitha/Data_Analytics/blob/main/masterNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analysis of Rain in Australia
This notebook contains all the steps of the analysis: From data cleaning and preprocessing to the conclusions.

In [ ]:
#Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats

In [ ]:
# Read the dataset and output the total count
df = pd.read_csv('/content/weatherAUS.csv')
print(df.head())
print(f"Total rows count: {df['RainToday'].count()}")

In [ ]:
# Check for the missing values
missing_values_count = df.isnull().sum()
print(missing_values_count)

# Calculate the percent of data that is missing
total_cells = np.prod(df.shape)
total_missing = missing_values_count.sum()

percent_missing = (total_missing/total_cells) * 100
print(f"Percentage of missing values: {percent_missing}")

In [ ]:
# Remove all the rows that contain a missing value
df = df.dropna()
#print(df.head())

# Check the remaining value count
print(f"Total rows Left: {df['RainToday'].count()}")